In [33]:
# Import các thư viện
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore

# Đường dẫn thư mục
data_dir = "../data/old_data/"
processed_dir = os.path.join(data_dir, "processed")

# Tạo thư mục lưu trữ dữ liệu nếu chưa có
os.makedirs(processed_dir, exist_ok=True)

# Danh sách các sàn giao dịch
exchanges = ["binance", "bittrex", "kraken", "kucoin", "lbank"]


In [34]:
# Hợp nhất dữ liệu từ các file CSV
all_data = []

for exchange in exchanges:
    exchange_dir = os.path.join(data_dir, "raw", exchange)
    files = [f for f in os.listdir(exchange_dir) if f.endswith(".csv")]

    for file in files:
        file_path = os.path.join(exchange_dir, file)
        df = pd.read_csv(file_path)
        
        # Lấy thông tin cặp giao dịch từ tên file
        pair = file.split("_")[1]
        
        # Thêm cột thông tin
        df["Pair"] = pair
        df["Exchange"] = exchange
        all_data.append(df)

# Hợp nhất toàn bộ dữ liệu
data = pd.concat(all_data, ignore_index=True)
print("Dữ liệu ban đầu:")
print(data.head())


Dữ liệu ban đầu:
   Unnamed: 0            Timestamp      Open      High       Low     Close  \
0           0  2018-04-20 00:00:00  0.032400  0.032536  0.032249  0.032295   
1           1  2018-04-20 01:00:00  0.032377  0.032399  0.031879  0.032141   
2           2  2018-04-20 02:00:00  0.032140  0.032733  0.031999  0.032576   
3           3  2018-04-20 03:00:00  0.032628  0.032701  0.032290  0.032290   
4           4  2018-04-20 04:00:00  0.032290  0.032374  0.031880  0.032105   

    Volume     Pair Exchange  
0  1387.89  ETC-ETH  binance  
1  2582.66  ETC-ETH  binance  
2  3393.88  ETC-ETH  binance  
3   959.78  ETC-ETH  binance  
4  2508.01  ETC-ETH  binance  


In [35]:
# Chuyển Timestamp sang datetime
data["Timestamp"] = pd.to_datetime(data["Timestamp"])

# Loại bỏ giá trị trùng lặp
data.drop_duplicates(subset=["Timestamp", "Pair", "Exchange"], inplace=True)

# Sắp xếp dữ liệu
data.sort_values(by="Timestamp", inplace=True)
print(data.info())


<class 'pandas.core.frame.DataFrame'>
Index: 468955 entries, 0 to 449760
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  468955 non-null  int64         
 1   Timestamp   468955 non-null  datetime64[ns]
 2   Open        454238 non-null  float64       
 3   High        454238 non-null  float64       
 4   Low         454236 non-null  float64       
 5   Close       454238 non-null  float64       
 6   Volume      468955 non-null  float64       
 7   Pair        468955 non-null  object        
 8   Exchange    468955 non-null  object        
dtypes: datetime64[ns](1), float64(5), int64(1), object(2)
memory usage: 35.8+ MB
None


In [36]:
# Kiểm tra giá trị thiếu
missing_values = data.isnull().sum()
print("Missing values:\n", missing_values)

# Điền giá trị thiếu bằng phương pháp Linear Interpolation
data.interpolate(method="linear", inplace=True)

# Xác nhận lại không còn giá trị thiếu
print("Missing values after filling:\n", data.isnull().sum())


Missing values:
 Unnamed: 0        0
Timestamp         0
Open          14717
High          14717
Low           14719
Close         14717
Volume            0
Pair              0
Exchange          0
dtype: int64
Missing values after filling:
 Unnamed: 0    0
Timestamp     0
Open          0
High          0
Low           0
Close         0
Volume        0
Pair          0
Exchange      0
dtype: int64


/var/folders/rz/mr63129s7vs6092h_t0czm140000gn/T/ipykernel_3941/2778096254.py:6: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data.interpolate(method="linear", inplace=True)


In [37]:
# Tính toán các đặc trưng mới
data["Price_Change_Pct"] = (data["Close"] - data["Open"]) / data["Open"] * 100
data["Volume_Spike"] = data["Volume"] / data["Volume"].rolling(window=10).mean()
data["Volatility"] = data["High"] - data["Low"]

# Điền NaN trong cột mới (do rolling tạo ra)
data.fillna(0, inplace=True)


print("Dữ liệu với đặc trưng mới:")
print(data[["Price_Change_Pct", "Volume_Spike", "Volatility"]].head())


Dữ liệu với đặc trưng mới:
        Price_Change_Pct  Volume_Spike    Volatility
0              -0.324074           0.0  2.870000e-04
44640          -0.097640           0.0  1.140000e-06
337440          0.000000           0.0  3.000000e-07
337920          1.851852           0.0  3.000000e-05
44160          -2.173550           0.0  1.600000e-04


In [38]:
# Import các thư viện cần thiết
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
# Chọn các đặc trưng để phát hiện bất thường
features = ["Open", "High", "Low", "Close", "Volume", "Price_Change_Pct", "Volume_Spike", "Volatility"]

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data[features])

# Isolation Forest với contamination ~0.01 (tỷ lệ bất thường ~1%)
iso_forest = IsolationForest(contamination=0.01, random_state=42)
data['Anomaly_Score'] = iso_forest.fit_predict(data_scaled)

# Gắn nhãn: -1 là bất thường, 1 là bình thường
data['Label'] = (data['Anomaly_Score'] == -1).astype(int)

print("Phân phối nhãn bất thường sau Isolation Forest:")
print(data['Label'].value_counts())


Phân phối nhãn bất thường sau Isolation Forest:
Label
0    464266
1      4689
Name: count, dtype: int64


In [39]:
# Chuẩn hóa các cột
scaler = MinMaxScaler()

columns_to_scale = ["Open", "High", "Low", "Close", "Volume"]
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

print("Dữ liệu sau chuẩn hóa:")
print(data.head())


Dữ liệu sau chuẩn hóa:
        Unnamed: 0  Timestamp          Open          High           Low  \
0                0 2018-04-20  2.968710e-08  2.981171e-08  2.985985e-08   
44640            0 2018-04-20  5.611265e-11  5.626842e-11  5.581456e-11   
337440           0 2018-04-20  4.974454e-11  5.001026e-11  5.026829e-11   
337920           0 2018-04-20  1.484172e-09  1.511651e-09  1.499799e-09   
44160            0 2018-04-20  5.480045e-09  5.497445e-09  5.407188e-09   

               Close    Volume     Pair Exchange  Price_Change_Pct  \
0       2.959089e-08  0.000001  ETC-ETH  binance         -0.324074   
44640   5.605767e-11  0.000108  AST-BTC  binance         -0.097640   
337440  4.974454e-11  0.000059  LYM-ETH   kucoin          0.000000   
337920  1.511661e-09  0.000003  BTM-ETH   kucoin          1.851852   
44160   5.360930e-09  0.000052  ICX-ETH  binance         -2.173550   

        Volume_Spike    Volatility  Anomaly_Score  Label  
0                0.0  2.870000e-04            

In [40]:
# Lưu dữ liệu đã xử lý
processed_file = os.path.join(processed_dir, "data_processed.csv")
data.to_csv(processed_file, index=False)
print(f"Dữ liệu đã lưu tại: {processed_file}")


Dữ liệu đã lưu tại: ../data/old_data/processed/data_processed.csv
